In [41]:
import pandas as pd
import numpy as np
import onnxruntime as ort

In [42]:
# Load dataset
data_path = "../../../data/synth_data_test_labeled.csv"  # Update to your dataset path
data = pd.read_csv(data_path)

# Load ONNX runtime model
model_path = "../../../model/model_1.onnx"  # Update to your ONNX model path
session = ort.InferenceSession(model_path)

In [43]:
# Configuration
feature_to_modify = "persoon_leeftijd_bij_onderzoek"
delta = 1  # Step size for hill climbing

In [44]:
# Helper function to make predictions
def predict_with_model(row):
    inputs = {session.get_inputs()[0].name: np.array(row).astype(np.float32).reshape(1, -1)}
    outputs = session.run(None, inputs)
    return outputs[1][0][0], outputs[1][0][1]  # Confidence for class 0 and class 1

In [45]:
# Randomly select a sample from the dataset
sample = data.sample(1).iloc[0]

In [46]:
original_value = sample[feature_to_modify]
original_row = sample.drop(['Ja', 'Nee'])  # Exclude the confidence columns
original_conf_0, original_conf_1 = predict_with_model(original_row)

print(f'Original feature value: {original_value}')
print(f'Original confidence: {original_conf_0, original_conf_1}')

Original feature value: 49.0
Original confidence: (0.5600000619888306, 0.4399999678134918)


In [47]:
# fitness function proposed by [A. Bartlett et al, SBFT 2023], outputs new confidence gap o1
def fitness_func(old_conf_0, old_conf_1, new_conf_0, new_conf_1):
    if old_conf_0 > old_conf_1 and new_conf_0 > new_conf_1:  # prediction not flipped
        return new_conf_0 - new_conf_1
    elif old_conf_1 > old_conf_0 and new_conf_1 > new_conf_0: # prediction not flipped
        return new_conf_1 - new_conf_0
    elif old_conf_0 > old_conf_1 and new_conf_1 > new_conf_0:
        return -1 * new_conf_1
    elif old_conf_1 > old_conf_0 and new_conf_0 > new_conf_1:
        return -1 * new_conf_0
    else:
        return ValueError

In [48]:
# Hillclimbing setup
old_conf_0, old_conf_1 = original_conf_0, original_conf_1
best_value = original_value
o1_original = fitness_func(original_conf_0, original_conf_1, original_conf_0, original_conf_1)
o1_current = o1_original
best_conf_0, best_conf_1 = original_conf_0, original_conf_1

In [49]:
no_improvement_iterations = 0

# stop hill climb if o1 has not improved for more than 10 iterations
while no_improvement_iterations < 10:
    delta_add = original_row.copy()
    delta_minus = original_row.copy()
    delta_add[feature_to_modify] += delta
    delta_minus[feature_to_modify] -= delta
    
    print('----------------------------------')
    print(f'Current value: {best_value}')
    print(f'Exploring neighbors: {delta_minus[feature_to_modify], delta_add[feature_to_modify]}')

    # get model predictions with modified feature value
    conf_0_add, conf_1_add = predict_with_model(delta_add) 
    conf_0_minus, conf_1_minus = predict_with_model(delta_minus)
    
    # calculate o1 (difference of confidence between two classes) for new predictions
    o1_add = fitness_func(original_conf_0, original_conf_1, conf_0_add, conf_1_add)
    o1_minus = fitness_func(original_conf_0, original_conf_1, conf_0_minus, conf_1_minus)
    
    print(f'o1_current: {o1_current}')
    
    # if original value plus delta gives lowest o1
    if o1_add < o1_current and o1_add < o1_minus:
        old_conf_0, old_conf_1 = conf_0_add, conf_1_add
        o1_current = o1_add
        best_value = delta_add[feature_to_modify]
        original_row = delta_add
        best_conf_0, best_conf_1 = conf_0_add, conf_1_add
        print(conf_0_add, conf_1_add)
        print(f'Improvement with modified value {best_value}. o1: {o1_current}')
        no_improvement_iterations = 0
    # if original value minus delta gives lowest o1
    elif o1_minus < o1_current and o1_minus < o1_add:
        old_conf_0, old_conf_1 = conf_0_minus, conf_1_minus
        o1_current = o1_minus
        best_value = delta_minus[feature_to_modify]
        original_row = delta_minus
        best_conf_0, best_conf_1 = conf_0_minus, conf_1_minus
        print(conf_0_minus, conf_1_minus)
        print(f'Improvement with modified value {best_value}. o1: {o1_current}')
        no_improvement_iterations = 0
    # if value from last iteration gives lowest o1
    else:
        print(best_conf_0, best_conf_1)
        print('No improvement detected')
        delta += 1  # increase hill-climb rate when stuck
        no_improvement_iterations += 1

----------------------------------
Current value: 49.0
Exploring neighbors: (48.0, 50.0)
o1_current: 0.12000009417533875
0.5500000715255737 0.44999995827674866
Improvement with modified value 48.0. o1: 0.10000011324882507
----------------------------------
Current value: 48.0
Exploring neighbors: (47.0, 49.0)
o1_current: 0.10000011324882507
0.5500000715255737 0.44999995827674866
No improvement detected
----------------------------------
Current value: 48.0
Exploring neighbors: (46.0, 50.0)
o1_current: 0.10000011324882507
0.5400000810623169 0.4599999487400055
Improvement with modified value 46.0. o1: 0.0800001323223114
----------------------------------
Current value: 46.0
Exploring neighbors: (44.0, 48.0)
o1_current: 0.0800001323223114
0.5000000596046448 0.4999999403953552
Improvement with modified value 44.0. o1: 1.1920928955078125e-07
----------------------------------
Current value: 44.0
Exploring neighbors: (42.0, 46.0)
o1_current: 1.1920928955078125e-07
0.4800000786781311 0.519999

In [50]:
# Output results
print(f"Original confidence: Class 0 = {original_conf_0}, Class 1 = {original_conf_1}")
print(f"Initial o1: {o1_original}")
print(f"Feature '{feature_to_modify}' changed from {original_value} to {best_value}")
print(f"Final confidence: Class 0 = {best_conf_0}, Class 1 = {best_conf_1}")
print(f"Optimized o1: {o1_current}")

Original confidence: Class 0 = 0.5600000619888306, Class 1 = 0.4399999678134918
Initial o1: 0.12000009417533875
Feature 'persoon_leeftijd_bij_onderzoek' changed from 49.0 to 31.0
Final confidence: Class 0 = 0.46000009775161743, Class 1 = 0.5399999022483826
Optimized o1: -0.5399999022483826
